In [5]:
import requests
import joblib
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from pandas import option_context
from tqdm import tqdm
from statsmodels.stats.inter_rater import aggregate_raters
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mannwhitneyu,shapiro,ttest_ind

import cliffsDelta as cd
import GenerateActivities as gat
import important_features as imf

In [ ]:
QUERY_ROOT = "https://api.github.com"
TOKEN = '' # write your GitHub API key here
HEADERS={'Authorization':'token '+TOKEN}

## Section 3

### Question 1

#### Write an automated Python script to identify all GitHub bot actors in the list of contributors, by querying the GitHub REST API users endpoint and extracting the required information to make this decision. 

#### More information: You can search for the "type" key in the obtianed JSON result, If it is "Bot", then it is a bot actor, if it is "User", then it is an account.

In [9]:
# 1. Read the given <sample>.csv file into a pandas DataFrame (df).
# 2. Make sure there are 2 columns, one column name is "contributor" that has the name of the contributor, 
#    and another column name is "bothunter_type" that you will use later on in this assignment.
# 3. Display the DataFrame.

import requests
import pandas as pd
import time
from tqdm import tqdm

# GitHub API URL template
GITHUB_API_URL = "https://api.github.com/users/{}"


# Load the dataset
csv_file = "Contributors/sample27.csv"  # Change this if needed
df = pd.read_csv(csv_file)

# Add a new column for GitHub API classification
df["github_api_type"] = ""

# Headers for authentication
headers = {"Authorization": f"token {TOKEN}"}

# Function to check if a user is a bot
def check_user_type(username):
    """Query GitHub API to determine if a user is a bot."""
    url = GITHUB_API_URL.format(username)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            user_data = response.json()
            return user_data.get("type", "Unknown")  # Returns "Bot" or "User"
        else:
            print(f"Failed to fetch {username}: {response.status_code}")
            return "Unknown"
    except Exception as e:
        print(f"Error fetching {username}: {e}")
        return "Unknown"

# Loop through contributors and classify them
for index, row in tqdm(df.iterrows(), total=len(df)):
    username = row["contributor"]
    df.at[index, "github_api_type"] = check_user_type(username)
    time.sleep(1)  # Avoid GitHub rate limits

# Save updated CSV
df.to_csv("sample27_with_actors.csv", index=False)

# Report total counts
bot_count = (df["github_api_type"] == "Bot").sum()
user_count = (df["github_api_type"] == "User").sum()
print(f"✅ Finished identifying bot accounts! {bot_count} Bots, {user_count} Users")


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:56<00:00,  1.42s/it]

✅ Finished identifying bot accounts! 7 Bots, 33 Users


In [21]:
# 1. Store the names of the contributors in a list
# 2. print the list

import pandas as pd

# 1. Store the names of the contributors in a list
contributors_df = pd.read_csv("Contributors/sample27.csv")
contributors_list = contributors_df["contributor"].tolist()

# 2. Print the list
print("Contributors list:")
print(contributors_list)

Contributors list:
['DanielVandH', 'Keno', 'PhilipYip1988', 'SylvainCorlay', 'adrn', 'allcontributors[bot]', 'arfon', 'bharatAmeria', 'charlesm93', 'davidpagnon', 'editorialbot', 'eriknw', 'github-actions[bot]', 'github-advanced-security[bot]', 'goanpeca', 'imagesc-bot', 'imincik', 'jgabry', 'juanitorduz', 'krishanbhasin-px', 'lumberbot-app[bot]', 'm-richards', 'mblue9', 'meeseeksmachine', 'mergify[bot]', 'mrocklin', 'nanosoldier', 'nstarman', 'pangeo-bot', 'petscbot', 'phcerdan', 'richard-burhans', 'ropensci-review-bot', 'scverse-bot', 'seberg', 'tcmetzger', 'transifex-integration[bot]', 'veenstrajelmer', 'welcome[bot]', 'zacharyburnett']


In [22]:
# 1. Using a for loop, iterate over each contributor in the list and query the GitHub Users API.
#    You can use "query = f'{QUERY_ROOT}/users/{contributor}'", where QUERY_ROOT is defined at the beginning of this notebook 
#    and 'contributor' is each individual contributor from the list
# 2. Get the response using 'response = requests.get(query, headers=HEADERS)'.
# 3. convert the response to JSON using 'json_response = response.json()'.
# 4. Iterate over each JSON response and get the value of the 'type' key. If it is "Bot" then the contributor is a bot actor, 
#    if "User" then the contributor is an account. You should boolean values to indicate if the contributor is a bot actor (True) or User/Organisation (False)
# 5. Save these results in list of dictionary of the form [{'contributor': <contributor name>, 'app': <boolean value>}, {...}, {...}, {...}].
#    Lets call this list as "app_dict"
# 6. Finally convert this list of dictionary to DataFrame by writing 'pd.DataFrame.from_dict(app_dict)'
# 7. Display the DataFrame. This should have two columns - contributor and app

import requests
import pandas as pd
import time

# Define constants (assuming these are defined at the beginning of the notebook)
QUERY_ROOT = "https://api.github.com"
HEADERS = {"Accept": "application/vnd.github.v3+json"}

# 1. Load the list of contributors
contributors_df = pd.read_csv("Contributors/sample27.csv")
contributors_list = contributors_df["contributor"].tolist()

# Create empty list to store results
app_dict = []

# 2-5. Iterate over each contributor and query the GitHub API
for contributor in contributors_list:
    # Construct query
    query = f'{QUERY_ROOT}/users/{contributor}'
    
    # Get response from GitHub API
    response = requests.get(query, headers=HEADERS)
    
    # Check if request was successful
    if response.status_code == 200:
        # Convert response to JSON
        json_response = response.json()
        
        # Check if the contributor is a bot based on the 'type' field
        # True if bot, False if user or organization
        is_bot = json_response.get('type') == "Bot"
        
        # Add to our list of dictionaries
        app_dict.append({
            'contributor': contributor,
            'app': is_bot
        })
    else:
        print(f"Error fetching data for {contributor}: {response.status_code}")
    
    # Add a small delay to avoid hitting API rate limits
    time.sleep(0.5)

# 6. Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame.from_dict(app_dict)

# 7. Display the DataFrame
print(result_df)

# Optionally save the results
result_df.to_csv("contributor_types.csv", index=False)

Error fetching data for veenstrajelmer: 403
Error fetching data for welcome[bot]: 403
Error fetching data for zacharyburnett: 403
                      contributor    app
0                     DanielVandH  False
1                            Keno  False
2                   PhilipYip1988  False
3                   SylvainCorlay  False
4                            adrn  False
5            allcontributors[bot]   True
6                           arfon  False
7                    bharatAmeria  False
8                      charlesm93  False
9                     davidpagnon  False
10                   editorialbot  False
11                         eriknw  False
12            github-actions[bot]   True
13  github-advanced-security[bot]   True
14                       goanpeca  False
15                    imagesc-bot  False
16                        imincik  False
17                         jgabry  False
18                    juanitorduz  False
19               krishanbhasin-px  False
20       

### Question 2

#### Add a new 'actor' column in the CSV file to store this information. You can write True if it is a bot actor and False if it is not.

#### Report on the total number of GitHub bot actors, and User accounts present in the list of accounts of your dataset.

In [26]:
# Merge the app DataFrame to df by writing 'pd.merge(df, <app df>, on='contributor'). This is similar to SQL join on primary key 'contributor'.
# The resultant df should have 3 columns - contributor, bothunter_type and user_type.

import pandas as pd

# 1️⃣ Load the contributor dataset (with BotHunter classification)
df = pd.read_csv("Contributors/sample27.csv")  # Contains 'contributor' and 'bothunter_type'

# 2️⃣ Load the bot classification dataset from the previous step
app_df = pd.read_csv("contributor_types.csv")  # Contains 'contributor' and 'app' (True/False)

# 3️⃣ Merge both DataFrames on 'contributor' (SQL-style join)
merged_df = pd.merge(df, app_df, on="contributor")

# 4️⃣ Rename 'app' column to 'actor' (to indicate bot actors)
merged_df.rename(columns={"app": "actor"}, inplace=True)

# 5️⃣ Save the updated dataset
merged_df.to_csv("merged_contributors.csv", index=False)

# 6️⃣ Count and report the number of bot actors and user accounts
bot_count = merged_df["actor"].sum()  # Sum of True values gives bot count
user_count = len(merged_df) - bot_count  # Total rows minus bot count gives user count

# 7️⃣ Display results
print(f"✅ Merging completed! Total Bots: {bot_count}, Total Users: {user_count}")
print(merged_df.head())  # Display first few rows


✅ Merging completed! Total Bots: 6, Total Users: 31
     contributor bothunter_type  actor
0    DanielVandH          Human  False
1           Keno          Human  False
2  PhilipYip1988          Human  False
3  SylvainCorlay          Human  False
4           adrn          Human  False


### Question 3

#### For each bot actor, write their purpose, the task that they automate by looking at their GitHub profile, homepage, information on the GitHub Marketplace, or other documentation that you can find online.


In [27]:
import requests
import pandas as pd
import time

# GitHub API setup
HEADERS = {"Authorization": f"token {TOKEN}", "Accept": "application/vnd.github.v3+json"}
QUERY_ROOT = "https://api.github.com"

# Load the merged contributor dataset
merged_df = pd.read_csv("merged_contributors.csv")

# Filter only bot actors
bot_actors = merged_df[merged_df["actor"] == True]["contributor"].tolist()

# Function to fetch GitHub events
def fetch_bot_activity(bot_username):
    url = f"{QUERY_ROOT}/users/{bot_username}/events"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code == 200:
        events = response.json()
        event_types = [event["type"] for event in events[:5]]  # Get the latest 5 event types
        return ", ".join(set(event_types))  # Remove duplicates
    else:
        return "Error fetching data"

# Create a list of bot activity
bot_details = []
for bot in bot_actors:
    activity = fetch_bot_activity(bot)
    bot_details.append({"bot": bot, "recent_activity": activity})
    time.sleep(1)  # Avoid rate limits

# Convert to DataFrame and display results
bot_df = pd.DataFrame(bot_details)
print(bot_df)

# Save bot details
bot_df.to_csv("bot_purpose_analysis.csv", index=False)
print("✅ Bot purpose analysis saved!")


                             bot  \
0           allcontributors[bot]   
1            github-actions[bot]   
2  github-advanced-security[bot]   
3             lumberbot-app[bot]   
4                   mergify[bot]   
5     transifex-integration[bot]   

                                     recent_activity  
0  CreateEvent, PullRequestEvent, IssueCommentEve...  
1                                          PushEvent  
2  PullRequestReviewCommentEvent, PullRequestRevi...  
3                                  IssueCommentEvent  
4           CreateEvent, PullRequestEvent, PushEvent  
5           CreateEvent, PullRequestEvent, PushEvent  
✅ Bot purpose analysis saved!


## Section 4

### Question 1

#### Write an automated script to use BIMBAS (Bot Identification Model Based on Activity Sequences) to obtain a prediction of the GitHub contributor type (Bot or Human) for the contributors present in your dataset.

#### Folow the steps given in each cell to use BIMBAS and obtain predictions.

### Read all the events and select the events performed by the contributors present in the given set

In [31]:
import pandas as pd
import joblib
from tqdm import tqdm

# 1️⃣ Load the BIMBAS model
try:
    bimbas_model = joblib.load("bimbas.joblib")
    print("✅ BIMBAS model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading BIMBAS model: {e}")
    exit()

# 2️⃣ Load the contributor dataset
csv_file = "Contributors/sample27.csv"  # Ensure correct path
try:
    df = pd.read_csv(csv_file)
    print(f"✅ Loaded {len(df)} contributors from {csv_file}")
except Exception as e:
    print(f"❌ Error loading CSV file: {e}")
    exit()

# 3️⃣ Check input format
if "contributor" not in df.columns:
    print("❌ 'contributor' column missing from dataset!")
    exit()

# 4️⃣ Function to predict contributor type using BIMBAS
def predict_bimbas(contributor):
    """Use BIMBAS model to predict if a contributor is a Bot or Human."""
    try:
        prediction = bimbas_model.predict([[contributor]])  # Ensure correct format
        return "Bot" if prediction[0] == 1 else "Human"
    except Exception as e:
        print(f"❌ Error predicting for {contributor}: {e}")
        return "Unknown"

# 5️⃣ Apply BIMBAS classification with a progress bar
df["BIMBAS"] = df["contributor"].progress_apply(predict_bimbas)

# 6️⃣ Save results
output_file = "bimbas_predictions.csv"
df.to_csv(output_file, index=False)
print(f"✅ BIMBAS classification completed and saved as '{output_file}'!")


✅ BIMBAS model loaded successfully!
✅ Loaded 40 contributors from Contributors/sample27.csv


AttributeError: 'Series' object has no attribute 'progress_apply'

### Execute BIMBAS to obtain the predictions

In [5]:
# Do not modify this cell
# Use the following function the code below to obtain your predictions

def execute_bimbas(contributors_list, selected_events):
    '''
    args: contributors_list (list) - list of contributors in the dataset provided to you
          selected_events (DataFrame) - DataFrame of events performed by the considered contributors
    return: bimbas_prediction (DataFrame) - DataFrame of preditions along with the confidence in prediction for each contributor
    '''
    result=pd.DataFrame()
    temp_list = []
    bimbas = joblib.load('bimbas.joblib')
    date_limit = pd.to_datetime(selected_events.created_at.max()) + pd.DateOffset(-90)
    
    for contributor in tqdm(contributors_list):
        # c = contributors[contributor]
        activities = gat.activity_identification(selected_events.query('login==@contributor and created_at>=@date_limit'))
        activity_features = (
                            imf.extract_features(activities)
                            .set_index([[contributor]])
                            )
        # features = pd.concat([features,activity_features])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            probability = bimbas.predict_proba(activity_features)[0][1]
        if(probability <= 0.5):
            contributor_type = 'Human'
        else:
            contributor_type = 'Bot'
        confidence = (abs(probability - 0.5)*2).round(3)
    
        pred = activity_features.set_index([[contributor]]).assign(
            prediction=contributor_type,
            confidence = confidence,
            )
        temp_list.extend([{'contributor':contributor,'bimbas_type':contributor_type,'confidence':confidence}])
    
    bimbas_prediction = pd.DataFrame.from_dict(temp_list)

    return(bimbas_prediction)

In [7]:
# Write your code here to call execute_bimbas function. 
# Pass the contributors list and the filtered events DataFrame as argument to "execute_bimbas" function
# Prediction returned by BIMBAS will have "contributor", "bimbas_type" and "confidence"
# Note: Print the prediction provided by BIMBAS - else this cell will not be graded.

# YOUR CODE BELOW

### Create a column in your predictions DataFrame to have predictions provided by BIMBAS. 
### Now your DataFrame should have the following columns - contributor, bothunter_type, app, bimbas_type, confidence 

### Merging bothunter and bimbas predictions

In [8]:
# Merge the predictions provided by BIMBAS to the resultant dataset of Section 3. This final DataFrame should have the following columns - 
# contributor, bothunter_type, user_type, bimbas_type, and confidence.

### Question 2
#### Using Cohen's Kappa compute and report the interrater agreement score between the labels computed by both the bot identification approaches. Mention your interpretation of Cohen's Kappa

In [10]:
# Hint: use the cohen_kappa_score library
# YOUR CODE BELOW

### Question 3
#### Determine the final type of each contributor. Whenever user_type column has the value "User" check if both bimbas_type and bothunter_type give the same prediction, then consider it as your final prediction in "acc_type" column.
#### For the contributors that have different predictions, i.e. bimbas_type is not same as bothunter_type, then make a manual verification and consider that as the type for that contributor. Add an extra column to the DataFrame ﬁle called 'manual' that has the label determined by you. 
#### For manual verification you can make use the GitHub UI and the GitHub API https://api.github.com/users/username

#### Whenever user_type column has the value "Bot", directly write your final acc_type as "Bot Actor"

#### More information:
For each contributor, you can look at their activities in GitHub UI, look at their latest events using GitHub Events API - https://api.github.com/users/<contributor>/events, to make a decision on their type.

In addition, you can also write a very small reason why do you think they are bot or human (e.g., same activity at regular interval so Bot, files committed and code modified looks like Human, comments look like Human, comment look like Bot, test report looks automated so Bot, and so on....). This will be useful to answer your next question.

In [11]:
# Write the code for your manual decision here
# example : 
# manual_list_dict = [{'contributor': '<contributor name>', 'manual': '<your prediction>'}, {....}, {....}, {....}]
# pd.DataFrame.from_dict(manual_list_dict)

# YOUR CODE BELOW


In [12]:
# Write your code below here to determine the final type of contributor.
# add column named 'acc_type' and write your final decision in it. Final decision is majority of three types - bimbas_type, bothunter_type, manual.


### Question 4
#### Study and report the purpose of these identified bots and humans in the repository assigned to you.
#### For example, the purpose can be based on but not limited to - 
##### 1) the type of activities that they are performing (releasing a version on every Sunday, updating the documentation), 
##### 2) type of comments they are posting (reviewing code, test summary report), 
##### 3) when are they triggered (when a new PR is created, when someone in the project ask the bot to merge the code) and so on.  
#### Note: 
##### The purpose can be other than what is provided in examples above.

## Section 5

### Question 1
#### Use the filtered events file that has the events performed by the contributors provided to you
#### Group each event into the following four categories

Issues: IssueCommentEvent, IssuesEvent  
Pull Requests: PullRequestEvent, PullRequestReviewCommentEvent  
Commits: CommitCommentEvent, PushEvent  
Repository: CreateEvent, DeleteEvent, ForkEvent, GollumEvent, MemberEvent, PublicEvent, ReleaseEvent, SponsorshipEvent, WatchEvent  

#### Hint:
1. Add a column called event_group that mentions which event group does that event belong to. Each event (row) should correspond to an event group.  
2. Then perform groupby on ['login','event_group'],  
3. use .agg (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) to count the number of events performed by each contributor in each group,  
4. use pivot with the required arguments (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html). An example is given [here](#pivot_example) in question 2(b), and
5. Reset index and rename axis with None
7. Finally merge it with your 'acc_type' field from the DataFrame you created in Section4, drop contributor, and fillna with 0
8. The final DataFrame should have the following columns - login, PR, commit, issue, repo, acc_type

The final DataFrame should be looking like the following  
![](event_group.png)

In [18]:
# In the resultant DataFrame - each row should corespond to a contributor, and the columns should have all the event groups  
# and the type of the contributor (that you decided in the previous DataFrame) and the values should be the number 
# of events of that event event group the contriubutor has performed.

issue_group = ['IssueCommentEvent', 'IssuesEvent']
pr_group = ['PullRequestEvent', 'PullRequestReviewCommentEvent']
commit_group = ['CommitCommentEvent', 'PushEvent']
repo_group = ['CreateEvent', 'DeleteEvent', 'ForkEvent', 'GollumEvent', 'MemberEvent', 'PublicEvent', 'ReleaseEvent', 'SponsorshipEvent', 'WatchEvent']

In [19]:
# YOUR CODE BELOW


In [20]:
# YOUR CODE BELOW


### Question 2 (a)

#### Compute the median number of events per event group for Bot+Bot actors and Humans and write in DataFrame.

Row should correspond to type (Bot_BotActor and Human), Column should have Event group name and the values should be the median value of Bot_BotActor or Human for that particular event group. An example is given below

In [15]:
# For example:
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot_app'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot_app'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot_app
1,event_group1,val2,Human
2,event_group2,val3,Bot_app
3,event_group2,val4,Human
4,event_group3,val5,Bot_app
5,event_group3,val6,Human
6,event_group4,val7,Bot_app
7,event_group5,val8,Human


In [22]:
# YOUR CODE BELOW

### Question 2 (b)

Plot a heatmap of the DataFrame using seaborn - 
1. First convert the dataframe to the required format using pivot, example is given below
2. plot using seaborn - sns.heatmap(df_medians, annot=True, vmin=0, vmax=300, cmap="crest"). More details: https://seaborn.pydata.org/generated/seaborn.heatmap.html)

#### pd.pivot example:
<a id='pivot_example'></a>

In [17]:
# Main DataFrame
medians = [{'event_group': 'event_group1', 'median': 'val1', 'acc_type': 'Bot'}, 
           {'event_group': 'event_group1', 'median': 'val2', 'acc_type': 'Human'},
           {'event_group': 'event_group2', 'median': 'val3', 'acc_type': 'Bot'},
           {'event_group': 'event_group2', 'median': 'val4', 'acc_type': 'Human'},
           {'event_group': 'event_group3', 'median': 'val5', 'acc_type': 'Bot'},
           {'event_group': 'event_group3', 'median': 'val6', 'acc_type': 'Human'},
           {'event_group': 'event_group4', 'median': 'val7', 'acc_type': 'Bot'},
           {'event_group': 'event_group5', 'median': 'val8', 'acc_type': 'Human'}]
df_medians = pd.DataFrame.from_dict(medians)
df_medians

,event_group,median,acc_type
0,event_group1,val1,Bot
1,event_group1,val2,Human
2,event_group2,val3,Bot
3,event_group2,val4,Human
4,event_group3,val5,Bot
5,event_group3,val6,Human
6,event_group4,val7,Bot
7,event_group5,val8,Human


In [18]:
# pivot the main DataFrame
df_medians.pivot(index='acc_type', columns='event_group', values='median')#.reset_index().rename_axis(None,axis=1)

event_group,event_group1,event_group2,event_group3,event_group4,event_group5
acc_type,,,,,
Bot,val1,val3,val5,val7,NaN
Human,val2,val4,val6,NaN,val8


In [25]:
# YOUR CODE BELOW

In [26]:
# YOUR CODE BELOW


#### What is the difference that you observe between Bots+Bot actors and Humans?

#### What is the differnce that you observe between Event groups?

#### What is the difference that you observe between Bots+Bot actors and Humans and Event groups all considering at the same time?

### Question 3

#### Create boxen plots to visualise the distribution of number of events in each event group. 
#### For more information you can visit - https://seaborn.pydata.org/generated/seaborn.boxenplot.html#seaborn.boxenplot 
#### You should highlight the data points that correspond to bots using a stripplot in seaborn. https://seaborn.pydata.org/tutorial/categorical.html#categorical-tutorial  
#### Interpret the results of the visualisation.

In [27]:
# YOUR CODE BELOW - Visualize number of events from pull request event group


In [28]:
# YOUR CODE BELOW - Visualise number of events from issue event group


In [29]:
# YOUR CODE BELOW - Visualise number of events from commit event group


In [30]:
# YOUR CODE BELOW - Visualise number of events from repo event group


### Question 4.1

#### Statistical identify whether number of events in each event group is normally distributed or not.
#### Null hypothesis - $H_0$: Sample comes from the data that has normal distribution.
#### Use Shapiro-Wilk test for this purpose. Use the p-value with a threshold of 0.05 to determine whether $H_0$ can be rejected with statistical significance or not. 

#### Use shapiro from scipy.stats to perform this test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

In [ ]:
# YOUR CODE BELOW

### Question 4.2
#### Determine if there is any statistical diﬀerence in number of events between the identified event groups. Perform this test on all pairs of event groups. 
#### Null hypothesis - $H_0$: Any two event group come from the same population.
#### If $H_0$ is rejected in Shapiro-Wilk test (at least for one of the two event groups considered for test), use the Mann-Whitney U statistical test for this purpose. If $H_0$ is not rejected in Shapiro-Wilk test, use the independent t-test for this purpose.  
#### In any case, use the p-value with a threshold of 0.01 to determine whether $H_0$ can be rejected with statistical significance. 

#### Use mannwhitneyu from scipy.stats to perform Mann-Whitney U test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) 
#### or 
#### ttest_ind from scipy.stats to perform independent t-test https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html.

#### More information:

You can pass the following arguments to mannwhitneyu from scipy.stats - (method='exact', nan_policy='omit'). For ttest_ind you can use - (nan_policy='omit')

In [31]:
# YOUR CODE BELOW

### Question 4.3
#### Each time you reject the null hypothesis $H_0$, quantify the effect size of the diﬀerence between the groups using cliﬀ’s delta ($\delta$). 
#### To calculate cliﬀ’s delta, you can pass the list of values to cliﬀ delta.py file given in the repository. E.g., cliffsDelta.cliffsDelta(list of values, list of values). This will return the effect size.
#### Refer to the table given in the TP document and mention your interpretation (negligible, small, medium, large).

In [ ]:
# YOU CODE BELOW